In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=ea20c04c864c516dda249d3b7b63f83bb8c00c3ac80a2e67d3e79ccfd52dd20d
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


Problema 1 - Diego Rodriguez - Joaquín Cabello

1.1
Problema 1. Programa el algoritmo de PageRank siguiendo la estrategia de arriba. En concreto, deberías seguir los siguientes pasos:

1. Prepara un RDD que tenga cada nodo con su PageRank inicial. Luego, haz una función que prepare el mensaje que cada nodo va a enviar. Probablemente quieras almacenar estos valores como otro RDD.

2. Escribe una función que se encargue del intercambio de mensajes entre nodos. Esta función envía los mensajes a los nodos correspondientes y se encarga del merge de los mensajes recibidos por cada nodo. Debe retornar un RDD que para cada nodo indique cuál es el mensaje final recibido.

3. Haz una función que actualice el valor de PageRank para cada nodo considerando el damping factor. Probablemente quieras hacer una función que tome el output del punto anterior y lo procese.

4. Itera los pasos correspondientes por un número máximo de iteraciones, o hasta que la diferencia entre dos iteraciones del valor de PageRank sea mínima.

El programa debe retornar cada nodo junto con su valor final de PageRank.

In [3]:
from pyspark.sql import SparkSession

In [4]:
# Set-up la sesión
spark = SparkSession.builder \
    .getOrCreate()

sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [4]:
nodes = [1, 2, 3, 4]
edges = [(1, 2), (2, 3), (2, 4), (3, 2)]

In [5]:
# Prepara un RDD que tenga cada nodo con su Page Rank inicial. Luego, haz una funci´on que prepare el
# mensaje que cada nodo va a enviar. Pprobablemente quieras almacenar estos valores como otro RDD

nodes_pagerank_RDD = sc.parallelize([(x, 1/len(nodes)) for x in nodes])
print('Nodos iniciales:',nodes_pagerank_RDD.collect())

def prep_mensaje(nodes_pagerank_RDD, edges, iter=None):
  edg = sc.parallelize(edges)
  num = edg.map(lambda x: (x[0], 1)).reduceByKey(lambda x,y: x+y) # Obtenemos la cantidad de mensajes enviados por nodo
  edg_ = edg.join(num)\
  .map(lambda x: (x[0], ((x[0], x[1][0]), x[1][1]))).join(nodes_pagerank_RDD)\
                        .map(lambda x: (x[1][0][0], x[1][1]/x[1][0][1] )) # Vemos quien manda el mensaje y cuantos\. Hacemos el calculo por mensaje

  return edg_

msg = prep_mensaje(nodes_pagerank_RDD, edges)
print('Mensajes enviados por nodo:', msg.collect())
msg = prep_mensaje(nodes_pagerank_RDD, edges)
print('Mensajes enviados por nodo:', msg.collect())


Nodos iniciales: [(1, 0.25), (2, 0.25), (3, 0.25), (4, 0.25)]
Mensajes enviados por nodo: [((1, 2), 0.25), ((2, 3), 0.125), ((2, 4), 0.125), ((3, 2), 0.25)]
Mensajes enviados por nodo: [((1, 2), 0.25), ((2, 3), 0.125), ((2, 4), 0.125), ((3, 2), 0.25)]


In [6]:
# Escribe una función que se encargue del intercambio de mensajes entre nodos.
# Esta función envía los mensajes a los nodos correspondientes y se encarga del merge de los mensajes
# recibidos por cada nodo. Debe retornar un RDD que para cada nodo indique cuál es el mensaje final recibido.

def merge_received_msg(nodes, msg):
  nodes = sc.parallelize(nodes)
  received = msg.map(lambda x: (x[0][1], x[1])).reduceByKey(lambda x,y: x + y)

  return received

r_msg = merge_received_msg(nodes, msg)
print('Merge mensajes recibidos por nodo:', r_msg.collect())

Merge mensajes recibidos por nodo: [(2, 0.5), (3, 0.125), (4, 0.125)]


In [7]:
# Haz una función que actualice el valor de PageRank para cada nodo considerando el damping factor.
# Probablemente quieras hacer una función que tome el output del punto anterior y lo procese.

def pagerank_node(d, nodes, r_msg):
  nodes = sc.parallelize(nodes).map(lambda x: (x, 0))
  pr = nodes.leftOuterJoin(r_msg).map(lambda x: (x[0], x[1][1] if x[1][1] != None else 0))
  n = (1-d)/nodes.count()
  pr = pr.map(lambda x: (x[0], (x[1] * d) + n ))
  return pr

d = 0.85
pr = pagerank_node(d, nodes, r_msg)
print(pr.collect())

[(1, 0.037500000000000006), (2, 0.4625), (3, 0.14375), (4, 0.14375)]


In [176]:
# Itera los pasos correspondientes por un n´umero m´aximo de iteraciones, o hasta que la diferencia entre
# dos iteraciones del valor de Page Rank sea m´ınima.
from math import sqrt


def page_rank_inador(nodos, edges, d, nodes_RDD=None, iter=1,max_iter=-1, prevdiff=0):

  if iter == 1:
    nodes_RDD =  sc.parallelize([(x, 1/len(nodes)) for x in nodes])
  else:
    nodes_RDD =  nodes_RDD

  print('Iteración', iter)
  iter +=1

  pr_n = pagerank_node(d, nodes, merge_received_msg(nodes, prep_mensaje(nodes_RDD, edges)))


  diff_rdd = nodes_RDD.join(pr_n).map(lambda x: ('key', x[1][1] - x[1][0])).map(lambda x: ('key',pow(x[1],2))).reduceByKey(lambda x, y: x + y).map(lambda x: sqrt(x[1]))


  # print("in->",nodes_RDD.collect())
  # print("out->",pr_n.collect())
  #print('Diff: ', diff_rdd.collect())

  # if diff_rdd.filter(lambda x: x < 0.00069).count() > 0:
  #   return pr_n

  if not max_iter == 1:
    max_iter-=1
    return page_rank_inador(nodos, edges, d, nodes_RDD=pr_n,iter=iter ,max_iter=max_iter, prevdiff=0)

  return pr_n

d = 0.85
pr_nodos = page_rank_inador(nodes, edges, d, max_iter=10)


Iteración 1
Iteración 2
Iteración 3
Iteración 4
Iteración 5
Iteración 6
Iteración 7
Iteración 8
Iteración 9
Iteración 10


In [177]:
pr_nodos.collect()

[(1, 0.037500000000000006),
 (2, 0.15907557964792862),
 (3, 0.10706817498735734),
 (4, 0.10706817498735734)]

La estrategia generalizada para el algortmo

In [31]:
def page_rank_general(grafo, d, epsilon,nodos=None, edges=None, nodes_RDD=None, iter=1,max_iter=-1, num_nodos=None):

  print('Iteracion', iter)
  # -------- Preparación de los nodos ---------#
  if iter == 1:
    nodos_RDD_inicial = sc.parallelize([(x, 1/len(grafo.nodes)) for x in grafo.nodes])
    nodos_RDD = sc.parallelize(grafo.nodes)
    edges_RDD = sc.parallelize(grafo.edges)
    num_nodos = nodos_RDD.count()
  else:
    nodos_RDD_inicial = nodos
    nodos_RDD = nodes_RDD
    edges_RDD = edges

  # -------------------------------------------#
  iter+= 1

  # -------- Preparamos los mensajes-----------#
  num = edges_RDD.map(lambda x: (x[0], 1)).reduceByKey(lambda x,y: x+y) # Obtenemos la cantidad de mensajes enviados por nodo
  edg_ = edges_RDD.join(num)\
  .map(lambda x: (x[0], ((x[0], x[1][0]), x[1][1]))).join(nodos_RDD_inicial)\
                        .map(lambda x: (x[1][0][0], x[1][1]/x[1][0][1] )) # Vemos quien manda el mensaje y cuantos\. Hacemos el calculo por mensaje
  # -------------------------------------------#

  # ----------- Merge de los mensajes ---------#
  received = edg_.map(lambda x: (x[0][1], x[1])).reduceByKey(lambda x,y: x + y)
  # -------------------------------------------#

  # ----------- Actualización de Nodos --------#
  nodos_vacio = nodos_RDD.map(lambda x: (x, 0))
  pr = nodos_vacio.leftOuterJoin(received).map(lambda x: (x[0], x[1][1] if x[1][1] != None else 0))
  n = (1-d)/num_nodos
  pr = pr.map(lambda x: (x[0], (x[1] * d) + n ))
  # -------------------------------------------#

  # ----------- Condición de término ----------#
  diff_rdd = nodos_RDD_inicial.join(pr).map(lambda x: ('key', x[1][1] - x[1][0])).map(lambda x: ('key',pow(x[1],2))).reduceByKey(lambda x, y: x + y).map(lambda x: sqrt(x[1]))
  if diff_rdd.filter(lambda x: x < epsilon).count() == 1:
    return pr

  # ----------- Condición max iteraciónes -------#
  if not max_iter == 1:
    max_iter-=1
    return page_rank_general(grafo, d, epsilon,nodos=pr, edges=edges_RDD, nodes_RDD=nodos_RDD, iter=iter,max_iter=max_iter, num_nodos=num_nodos)

  return pr

Cargamos el Grafo de Cora, utilizamos el codigo para cargar la información del grafo del archivo "12 - Actividad Evaluada Grafos.ipynb".

In [24]:
import pandas as pd
import networkx as nx
import numpy as np
from math import sqrt


In [6]:
### Información de las aristas del grafo de cora

cora_cites_path = '/content/drive/MyDrive/Universidad/Procesamiento de datos masivos/Actividades/12 - Actividad Evaluada Grafos/cora/cora/cora.cites'
cora_content_path = '/content/drive/MyDrive/Universidad/Procesamiento de datos masivos/Actividades/12 - Actividad Evaluada Grafos/cora/cora/cora.content'

### Información de las aristas del grafo de cora

citas = pd.read_csv(cora_cites_path,sep="\t",
    header=None,
    names=["target", "source"])

### Metemos esto a networkx

G = nx.from_pandas_edgelist(citas, source="source", target="target",create_using=nx.DiGraph())

### Ahora levantamos la información de cada paper. Solo vamos a usar el "subject"

### Información de los nodos de cora

column_names = ["paper_id"] + [f"word_{idx}" for idx in range(1433)] + ["subject"]
papers = pd.read_csv(
    cora_content_path, sep="\t", names=column_names,
)
subjects = papers[["paper_id","subject"]]

### Agregamos la información a nuestro grafo G de esta forma:

subjects_dict = dict(zip(subjects['paper_id'], subjects['subject']))
nx.set_node_attributes(G, subjects_dict, 'subject')

Hacemos la prueba con el grafo:

In [34]:
G = G
DAMPING_FACTOR = 0.85
EPSILON =  0.00069
MAX_ITERACIONES = 10

pr_grafo = page_rank_general(G, DAMPING_FACTOR, EPSILON, max_iter=MAX_ITERACIONES)

# PARA VISUALIZAR EL TOP 10
top = 10
print(f'El top {top} pagerank del grafo es:')
pr_grafo.top(top, key=lambda x: x[1])

Iteracion 1
Iteracion 2
Iteracion 3
Iteracion 4
Iteracion 5
Iteracion 6
Iteracion 7
Iteracion 8
Iteracion 9
Iteracion 10
El top 10 pagerank del grafo es:


[(15429, 0.01143136472116835),
 (35, 0.011229497231473954),
 (10177, 0.010936443783534672),
 (210871, 0.005358044814531468),
 (210872, 0.0044934464342326),
 (82920, 0.004004069686793562),
 (1365, 0.003583970434052023),
 (4584, 0.003560500448362012),
 (643221, 0.0032767621957340547),
 (887, 0.003275659826096156)]

La prueba del algoritmo con el ejemplo del PDF:

In [35]:
nodes = [1, 2, 3, 4]
edges = [(1, 2), (2, 3), (2, 4), (3, 2)]

G_0 = nx.DiGraph()
G_0.add_nodes_from(nodes)
G_0.add_edges_from(edges)


In [36]:
DAMPING_FACTOR = 0.85
EPSILON =  0.00069
MAX_ITERACIONES = 10
pr_grafo_0 = page_rank_general(G_0, DAMPING_FACTOR, EPSILON, max_iter=MAX_ITERACIONES)

# PARA VISUALIZAR EL TOP 10
top = 10
print(f'El top {top} pagerank del grafo es:')
pr_grafo_0.top(top, key=lambda x: x[1])

Iteracion 1
Iteracion 2
Iteracion 3
Iteracion 4
Iteracion 5
Iteracion 6
Iteracion 7
Iteracion 8
Iteracion 9
Iteracion 10
El top 10 pagerank del grafo es:


[(2, 0.15907557964792862),
 (3, 0.10706817498735734),
 (4, 0.10706817498735734),
 (1, 0.037500000000000006)]